In [3]:
from nltk.corpus import wordnet as wn
import pandas as pd
import numpy as np
import os
import numpy as np
import glob
import pickle
import nltk
import copy

In [4]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to C:\Users\Shiyuan
[nltk_data]     Wang\AppData\Roaming\nltk_data...


True

In [8]:
import os
os.chdir("C:/0-ece285/project")

# Merge label files


## imagenet

In [3]:
with open("BOLD5000_Stimuli/Image_Labels/imagenet_final_labels.txt") as f:
    imagenet_contents = f.readlines()

In [4]:
keys = [line.split(" ", 1)[0] for line in  imagenet_contents]
values = [line.split(" ", 1)[1].strip("\n") for line in  imagenet_contents]
imagenet_annot = dict(zip(keys, values))

## COCO

In [5]:
coco_contents = pd.read_pickle("BOLD5000_Stimuli/Image_Labels/coco_final_annotations.pkl")

In [6]:
coco_ids = pd.read_csv("BOLD5000_Stimuli/Image_Labels/coco-labels-paper.txt", names = ['annotation'])

In [7]:
image_ids = np.empty(len(coco_contents), dtype=object)
annotation = np.empty(len(coco_contents), dtype=object)
i = 0
for k in coco_contents.keys():
    image_ids[i] = str(k)
    length = len(coco_contents[k])
    annotation[i] = ", ".join(list(set([coco_ids.loc[coco_contents[k][j]['category_id'],'annotation'] for j in range(length)]) ))
    i += 1
coco_annot = dict(zip(image_ids, annotation))

In [62]:
CATEGORIES = ['person','animal','plant','food','artifact','event',"entity"]
length = len(glob.glob("stim_lists/CSI0*.txt"))
for i in range(length):
    f_name_stim = glob.glob("stim_lists/CSI0*.txt")[i]
    stim_list =  pd.read_csv(f_name_stim, names = ["names"])['names'].tolist()
    len_list = len(stim_list)
    annotation = np.empty(len_list, dtype=object)
    for i in range(len_list):
        image_id = stim_list[i]
        if 'COCO' in image_id:
            annotation[i] = coco_annot[image_id.split("_")[-1].lstrip("0").split(".jpg")[0]]
        elif image_id[0] == "n" and image_id[1] in '0123456789':
            annotation[i] = imagenet_annot[image_id.split("_")[0]]
        else:
            annotation[i] = image_id.split(".")[0].rstrip('0123456789')

    df_stim_annot = pd.DataFrame({"image_id":stim_list, "annotation": annotation})
    df_stim_annot['category'] = ""
    for i in range(len(df_stim_annot)):
        annot_list = df_stim_annot.loc[i, "annotation"].split(", ")


        found = False
        try:
            hyper_list = list(set([w for s in wn.synset(annot_list[j].replace(" ", "_") + '.n.01').closure(lambda s:s.hypernyms()) for w in s.lemma_names() for j in range(len(annot_list))]))
            for cat in CATEGORIES:
                if cat in hyper_list:
                    df_stim_annot.loc[i, 'category'] = cat
                    found = True
                    break
        except:
            df_stim_annot.loc[i, 'category'] = "entity"

        if found == False:
            df_stim_annot.loc[i, 'category'] = "entity"

    df_stim_annot.to_csv("stim_lists/annotation/" + f_name_stim.strip("stim_lists/").strip(".txt") + "_annotation.csv")

/opt/anaconda3/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:599: UserWarning: Discarded redundant search for Synset('instrumentality.n.03') at depth 4
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:599: UserWarning: Discarded redundant search for Synset('animal.n.01') at depth 11
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:599: UserWarning: Discarded redundant search for Synset('animal.n.01') at depth 10
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:599: UserWarning: Discarded redundant search for Synset('instrumentality.n.03') at depth 7
  for synset in acyclic_breadth_first(self, rel, depth):
/opt/anaconda3/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:599: UserWarning: Discarded redundant search for Synset('mo

In [5]:
length = len(glob.glob("stim_lists/CSI0*.txt"))
annotations = set()
for i in range(length):
    f_name_stim = glob.glob("stim_lists/CSI0*.txt")[i]
    df_stim_annot= pd.read_csv("stim_lists/annotation/" + f_name_stim.strip("stim_lists/").strip(".txt") + "_annotation.csv")
    annotations = annotations.union(set(df_stim_annot.annotation.unique().tolist()))
    

In [6]:
len(list(annotations))

2119

# Merge ROI data


In [64]:
ROI_files = glob.glob("BOLD5000_ROI/*")

In [137]:
df_all = pd.DataFrame()
for f in ROI_files:
    sub = f.split("CSI")[1].split("_")[0]

    ROI = f.split("_")[-1].strip(".npy")
    image_annotation = pd.read_csv('stim_lists/annotation/CSI0{}_stim_lists_annotation.csv'.format(sub))
    categories = image_annotation['category'].tolist()
    image_ids = image_annotation['image_id'].tolist()
    data = np.load(f)
    voxel_length = len(data[0])
    df = pd.DataFrame(data, columns = ["voxel#" + str(i) for i in range(voxel_length)])
    df['subject'] = sub
    df['ROI'] = ROI
    df['category'] = categories
    df['image_id'] = image_ids
    df = df.reindex(["subject","ROI","category",'image_id'] + df.columns[:-4].tolist(), axis=1)
    df_all = df_all.append(df.reset_index(drop=True), ignore_index=True)
    df_all = df_all.reset_index(drop=True)
    
df_all.to_csv("ROI_combined.csv", index = False)

/var/folders/rb/plyftz9162s2ht1yrxblj4sc0000gn/T/ipykernel_13052/1119886964.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df.reset_index(drop=True), ignore_index=True)
/var/folders/rb/plyftz9162s2ht1yrxblj4sc0000gn/T/ipykernel_13052/1119886964.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df.reset_index(drop=True), ignore_index=True)
/var/folders/rb/plyftz9162s2ht1yrxblj4sc0000gn/T/ipykernel_13052/1119886964.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df.reset_index(drop=True), ignore_index=True)
/var/folders/rb/plyftz9162s2ht1yrxblj4sc0000gn/T/ipykernel_13052/1119886964.py:17: FutureWarning: The frame.append method is deprecated and

In [138]:
df_all

,subject,ROI,category,image_id,voxel#0,voxel#1,voxel#2,voxel#3,voxel#4,voxel#5,...,voxel#512,voxel#513,voxel#514,voxel#515,voxel#516,voxel#517,voxel#518,voxel#519,voxel#520,voxel#521
0,3,RHLO,artifact,COCO_train2014_000000292845.jpg,0.808443,-0.117471,0.719994,0.176429,-1.736903,-0.128124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,RHLO,artifact,COCO_train2014_000000296474.jpg,-0.365091,-0.225545,1.199927,-0.333959,0.251640,-0.757534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,RHLO,artifact,COCO_train2014_000000340488.jpg,-0.699597,-0.290462,-0.020237,-0.815934,-0.518546,-0.518137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,RHLO,entity,COCO_train2014_000000072095.jpg,-0.083338,-0.030962,0.166299,-0.238146,-0.183312,-1.105681,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,RHLO,artifact,COCO_train2014_000000207833.jpg,0.190421,-0.099735,1.037397,1.326302,1.537906,0.176843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188695,2,LHOPA,entity,waitingroom2.jpg,-0.815062,-1.273379,-1.525220,-1.667525,-0.511259,-0.755437,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188696,2,LHOPA,artifact,COCO_train2014_000000224396.jpg,-1.174958,0.605982,0.282339,-0.129642,-0.080109,-0.748996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188697,2,LHOPA,artifact,COCO_train2014_000000539967.jpg,-1.086591,-0.092096,-1.156625,-0.892493,-0.710585,-0.935369,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188698,2,LHOPA,animal,n02085782_1848.JPEG,-0.866085,0.259445,-0.147186,-0.379703,-0.649797,0.506657,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
df_all[~df_all.isnull().any(axis=1)]

,subject,ROI,category,image_id,voxel#0,voxel#1,voxel#2,voxel#3,voxel#4,voxel#5,...,voxel#512,voxel#513,voxel#514,voxel#515,voxel#516,voxel#517,voxel#518,voxel#519,voxel#520,voxel#521
45140,3,LHEarlyVis,artifact,COCO_train2014_000000292845.jpg,1.238665,0.404795,0.977252,0.022144,-0.981377,0.215708,...,-1.742685,-0.269162,0.557405,1.158340,1.450740,-0.445768,0.125318,1.370077,0.726587,0.247191
45141,3,LHEarlyVis,artifact,COCO_train2014_000000296474.jpg,0.442644,1.240564,0.828025,-0.017332,0.351841,0.202542,...,-2.077565,0.117445,1.358723,-3.273300,-2.119016,-2.418595,-0.794622,-0.430744,-0.513666,0.963983
45142,3,LHEarlyVis,artifact,COCO_train2014_000000340488.jpg,-0.068887,-0.074017,0.400164,-0.210570,0.207279,-0.461544,...,-1.346401,-0.494141,0.283005,-2.749937,-1.702408,-1.513198,-0.602901,-0.752867,-0.591900,-0.318313
45143,3,LHEarlyVis,entity,COCO_train2014_000000072095.jpg,-1.302105,-1.338536,-0.599421,-0.207713,-0.747720,-1.049721,...,-1.726323,-1.152239,-1.617354,0.429230,0.817624,-0.765071,-0.198426,0.026484,-0.199513,-0.011028
45144,3,LHEarlyVis,artifact,COCO_train2014_000000207833.jpg,-0.841807,-0.990209,0.090210,-0.821574,-0.310257,0.081427,...,-0.797432,-0.761251,-1.168870,1.413718,0.153169,-1.591235,-0.645669,-1.252535,-0.690070,-1.256320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50389,3,LHEarlyVis,artifact,fireplace3.jpg,-0.776073,0.282886,0.214093,0.185558,-0.442678,1.142137,...,3.404620,1.698642,1.552568,1.426583,2.081854,1.955662,1.149646,0.680833,1.838321,0.217327
50390,3,LHEarlyVis,artifact,n04418357_1004.JPEG,-1.508506,-0.977600,-0.049696,-0.148813,0.077139,-0.251964,...,-1.476893,-1.771978,-1.193621,-0.798118,-2.237137,-1.866461,-1.289521,-0.379670,-1.447591,-0.419079
50391,3,LHEarlyVis,artifact,COCO_train2014_000000099580.jpg,-0.409303,-1.935925,-0.674582,-1.761568,-0.599777,-1.921910,...,0.176893,-0.813738,-0.345220,0.843469,0.695784,-0.507232,-0.770936,0.691815,0.242548,0.551236
50392,3,LHEarlyVis,person,COCO_train2014_000000497498.jpg,0.160381,1.699413,1.363919,1.705248,1.267206,0.429813,...,-1.320361,-1.022910,0.048365,-0.020689,-0.290544,-1.075228,-0.515207,-0.034746,-0.414340,0.011215


In [9]:
df_all = pd.read_csv("ROI_combined.csv")

In [10]:
df_all

,subject,ROI,category,image_id,voxel#0,voxel#1,voxel#2,voxel#3,voxel#4,voxel#5,...,voxel#512,voxel#513,voxel#514,voxel#515,voxel#516,voxel#517,voxel#518,voxel#519,voxel#520,voxel#521
0,3,RHLO,artifact,COCO_train2014_000000292845.jpg,0.808443,-0.117471,0.719994,0.176429,-1.736903,-0.128124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,RHLO,artifact,COCO_train2014_000000296474.jpg,-0.365091,-0.225545,1.199927,-0.333959,0.251640,-0.757534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,RHLO,artifact,COCO_train2014_000000340488.jpg,-0.699597,-0.290462,-0.020237,-0.815934,-0.518546,-0.518137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,RHLO,entity,COCO_train2014_000000072095.jpg,-0.083338,-0.030962,0.166299,-0.238146,-0.183312,-1.105681,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,RHLO,artifact,COCO_train2014_000000207833.jpg,0.190421,-0.099735,1.037397,1.326302,1.537906,0.176843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188695,2,LHOPA,entity,waitingroom2.jpg,-0.815062,-1.273379,-1.525220,-1.667525,-0.511259,-0.755437,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188696,2,LHOPA,artifact,COCO_train2014_000000224396.jpg,-1.174958,0.605982,0.282339,-0.129642,-0.080109,-0.748996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188697,2,LHOPA,artifact,COCO_train2014_000000539967.jpg,-1.086591,-0.092096,-1.156625,-0.892493,-0.710585,-0.935369,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188698,2,LHOPA,animal,n02085782_1848.JPEG,-0.866085,0.259445,-0.147186,-0.379703,-0.649797,0.506657,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df_all["ROI"] = df_all["ROI"].replace("RHRRSC","RHRSC")
df_all["ROI"] = df_all["ROI"].replace("LHLO","LHLOC")
df_all["ROI"] = df_all["ROI"].replace("RHLO","RHLOC")

In [63]:
df_all.to_csv("ROI_combined.csv")

In [147]:
df_all[['subject','ROI']].value_counts()

subject  ROI       
1        LHEarlyVis    5254
2        RHLO          5254
3        RHRSC         5254
         RHPPA         5254
         RHOPA         5254
         RHLO          5254
         RHEarlyVis    5254
         LHRSC         5254
         LHPPA         5254
         LHOPA         5254
         LHLO          5254
1        LHLOC         5254
2        RHRRSC        5254
         RHPPA         5254
         RHOPA         5254
3        LHEarlyVis    5254
2        RHEarlyVis    5254
1        RHOPA         5254
         LHOPA         5254
2        LHRSC         5254
1        LHRSC         5254
         RHEarlyVis    5254
         RHLOC         5254
         LHPPA         5254
         RHPPA         5254
         RHRSC         5254
2        LHEarlyVis    5254
         LHLO          5254
         LHOPA         5254
         LHPPA         5254
4        LHEarlyVis    3108
         LHLOC         3108
         LHOPA         3108
         LHPPA         3108
         LHRSC         3108


In [89]:
ROIs = df_all.ROI.unique()
ROIs.sort()

for i in df_all.subject.unique():
    merged = []
    for j in df_all[df_all["subject"]==i].image_id.unique():
        

        data = [i, j]

        if len(df_all[(df_all["subject"]==i) & (df_all["image_id"]==j)].ROI.unique()) < len(ROIs):
            continue
        if len(merged) == 0:
            columns = ["subject", "image_id"]
        for ROI in ROIs:
            if len(merged) == 0:
                for k in range(len(df_all[(df_all["subject"]==i) & (df_all["image_id"]==j) & (df_all["ROI"]==ROI)].iloc[0, 4:].dropna())):
                    columns.append(ROI + "#"+  str(k))
           
            data = np.concatenate((data, df_all[(df_all["subject"]==i) & (df_all["image_id"]==j) & (df_all["ROI"]==ROI)].iloc[0, 4:].dropna().tolist()))
        merged.append(data)
        
        df_merged = pd.DataFrame(merged, columns = columns)
        df_merged.to_csv("df_merged_subject{}.csv".format(i), index = None)
            